In [ ]:
from grid.robot.aerial.airgen_drone import AirGenDrone 
airgen_drone__0 = AirGenDrone()

In [ ]:
# Take off
airgen_drone_0.client.takeoffAsync().join()

# Move to an altitude of 25 meters
airgen_drone_0.client.moveToZAsync(-25, 5).join()

In [ ]:
# Fly in a square path using velocity control

airgen_drone_0.client.moveByVelocityAsync(5, 0, 0, 5).join()
airgen_drone_0.client.moveByVelocityAsync(0, 5, 0, 5).join()
airgen_drone_0.client.moveByVelocityAsync(-5, 0, 0, 5).join()
airgen_drone_0.client.moveByVelocityAsync(0, -5, 0, 5).join()

In [ ]:
# Fly in a square path using position control

airgen_drone_0.client.moveToPositionAsync(25, 0, -25, 5).join()
airgen_drone_0.client.moveToPositionAsync(25, 25, -25, 5).join()
airgen_drone_0.client.moveToPositionAsync(0, 25, -25, 5).join()
airgen_drone_0.client.moveToPositionAsync(0, 0, -25, 5).join()

In [ ]:
# Fly in a square path using path control
import airgen
waypoints = [
    airgen.Vector3r(25, 0, -25),
    airgen.Vector3r(25, 25, -25),
    airgen.Vector3r(0, 25, -25),
    airgen.Vector3r(0, 0, -25),
]

airgen_drone_0.client.moveOnPathAsync(
    waypoints, 5, 60, airgen.DrivetrainType.MaxDegreeOfFreedom, airgen.YawMode(False, 0), -1, 1,
).join()

In [ ]:
# Fly in a square path with yaw angle relative to the direction of travel
airgen_drone_0.client.moveOnPathAsync(
    waypoints, 5, 60, airgen.DrivetrainType.ForwardOnly, airgen.YawMode(False, 0), -1, 1
).join()

In [ ]:
# Land
airgen_drone_0.client.moveToZAsync(-2, 5).join()
airgen_drone_0.client.landAsync().join()

In [ ]:
# plan a path in geo-typical environments
import numpy as np
nav_mesh_info = airgen_drone_0.client.getNavMeshInfo()
# Get current pose of the drone in 6-DOF
start_pose = airgen_drone_0.client.simGetVehiclePose()

# Sample a random valid pose in the environment
amplitude = (
    np.absolute(
        np.array(
            [
                nav_mesh_info[2]["x_val"] - nav_mesh_info[1]["x_val"],
                nav_mesh_info[2]["y_val"] - nav_mesh_info[1]["y_val"],
                nav_mesh_info[2]["z_val"] - nav_mesh_info[1]["z_val"],
            ]
        )
    )
    / 2.0
)
# sample a random point on the nav mesh
random_point = airgen.Vector3r(
    np.random.uniform(
        nav_mesh_info[0]["x_val"] - amplitude[0],
        nav_mesh_info[0]["x_val"] + amplitude[0],
    ),
    np.random.uniform(
        nav_mesh_info[0]["y_val"] - amplitude[1],
        nav_mesh_info[0]["y_val"] + amplitude[1],
    ),
    np.random.uniform(
        nav_mesh_info[0]["z_val"] - amplitude[2],
        nav_mesh_info[0]["z_val"] + amplitude[2],
    ),
)
# sample a random yaw angle
random_yaw = np.random.uniform(-np.pi, np.pi)
goal_pose= airgen.Pose(
            random_point, airgen.Quaternionr(airgen.Vector3r(0, 0, random_yaw))
        )

# Compute a collision-free path between start and goal
trajectory = airgen_drone_0.client.simPlanPath(
    start_pose.position, goal_pose.position, True, True
)

points = []
for waypoint in trajectory:
    points.append(
        airgen.Vector3r(waypoint["x_val"], waypoint["y_val"], waypoint["z_val"])
    )

# Move the drone along the planned path at a velocity of 5 m/s
velocity = 5.0
airgen_drone_0.client.moveOnPathAsync(
    points,
    velocity,
    120,
    airgen.DrivetrainType.ForwardOnly,
    airgen.YawMode(False, 0),
    -1,
    0,
).join()